# Run autoRIFT for Sentinel 2 images

In [1]:
# Import required packages
import os
import sys
import numpy as np
import matplotlib.pyplot as plt
from osgeo import gdal
from datetime import datetime, timedelta
import time
import rasterio as rio
import xarray as xr
import rioxarray
import isce
import logging
root_logger = logging.getLogger()
root_logger.setLevel('WARNING')
import logging
from imageMath import IML
import isce
from components.contrib.geo_autoRIFT.autoRIFT import autoRIFT_ISCE
from components.contrib.geo_autoRIFT.autoRIFT import __version__ as version
import isceobj
import subprocess
from scipy.interpolate import interpn
from glob import glob
import pandas as pd

In [2]:
os.environ['AUTORIFT'] = '/mnt/Backups/gbrench/sw/insar_tools/isce2/src/isce2/contrib/geo_autoRIFT'
os.environ['PYTHONPATH'] = os.environ['AUTORIFT']
os.environ['PATH'] += f":{os.environ['AUTORIFT']}"

In [3]:
# la valette landslide scene list:
#scene_list = os.listdir('data/OIC_mount_blanc_massif_glacier')
scene_list = ['T32TLR_20200812T103031_B08.tif',
 'T32TLR_20200916T102649_B08.tif',
 'T32TLR_20210812T102559_B08.tif']

In [4]:
proc_path = '/mnt/Backups/gbrench/repos/fusits/nbs/oic_challenge/data/OIC_mount_blanc_massif_glacier'

In [5]:
def select_pairs(scene_list, min_temp_bline, max_temp_bline):
    scene_dates = {}
    for scene in scene_list:
        date = scene[7:15]
        scene_dates[date] = scene

    pairs = []
    for date1 in scene_dates:
        for date2 in scene_dates:
            if datetime.strptime(date2, '%Y%m%d')-datetime.strptime(date1, '%Y%m%d') < timedelta(days=max_temp_bline) and not datetime.strptime(date2, '%Y%m%d')-datetime.strptime(date1, '%Y%m%d') < timedelta(days=min_temp_bline) and not date1 >= date2 :
                pairs.append([date1, date2])
    
    print(f'number of pairs: {len(pairs)}')
    
    return pairs

In [6]:
pairs = select_pairs(scene_list, 10, 100000) # all possible pairs

number of pairs: 3


In [7]:
def run_autoRIFT(pair, skip_x=1, skip_y=3, scale_y_chip=3, min_azm_chip=16, max_azm_chip=64,
                 preproc_filter_width=21, mpflag=10, search_limit_x=10, search_limit_y=10):

    print(f'opening pair: {pair[0]}-{pair[1]}')

    reference_fn = glob(f'{proc_path}/*{pair[0]}*.tif')[0]
    secondary_fn = glob(f'{proc_path}/*{pair[1]}*.tif')[0]

    ds1 = gdal.Open(reference_fn, gdal.GA_ReadOnly)
    img1 = ds1.GetRasterBand(1).ReadAsArray()

    ds2 = gdal.Open(secondary_fn, gdal.GA_ReadOnly)
    img2 = ds2.GetRasterBand(1).ReadAsArray()

    I1 = np.abs(img1)
    I2 = np.abs(img2)
        
    obj = autoRIFT_ISCE()
    obj.configure()
    obj.MultiThread = mpflag

    obj.I1 = I1
    obj.I2 = I2

    obj.SkipSampleX = skip_x
    obj.SkipSampleY = skip_y

    # Kernel sizes to use for correlation
    obj.ChipSizeMinX = min_azm_chip
    obj.ChipSizeMaxX = max_azm_chip
    obj.ChipSize0X = min_azm_chip
    # oversample ratio, balancing precision and performance for different chip sizes
    obj.OverSampleRatio = {obj.ChipSize0X:16,obj.ChipSize0X*2:32,obj.ChipSize0X*4:64}

    # generate grid
    m,n = obj.I1.shape
    xGrid = np.arange(obj.SkipSampleX+10,n-obj.SkipSampleX,obj.SkipSampleX)
    yGrid = np.arange(obj.SkipSampleY+10,m-obj.SkipSampleY,obj.SkipSampleY)
    nd = xGrid.__len__()
    md = yGrid.__len__()
    obj.xGrid = np.int32(np.dot(np.ones((md,1)),np.reshape(xGrid,(1,xGrid.__len__()))))
    obj.yGrid = np.int32(np.dot(np.reshape(yGrid,(yGrid.__len__(),1)),np.ones((1,nd))))
    noDataMask = np.invert(np.logical_and(obj.I1[:, xGrid-1][yGrid-1, ] > 0, obj.I2[:, xGrid-1][yGrid-1, ] > 0))

    # set search limits
    obj.SearchLimitX = np.full_like(obj.xGrid, search_limit_x)
    obj.SearchLimitY = np.full_like(obj.xGrid, search_limit_y)

    # set search limit and offsets in nodata areas
    obj.SearchLimitX = obj.SearchLimitX * np.logical_not(noDataMask)
    obj.SearchLimitY = obj.SearchLimitY * np.logical_not(noDataMask)
    obj.Dx0 = obj.Dx0 * np.logical_not(noDataMask)
    obj.Dy0 = obj.Dy0 * np.logical_not(noDataMask)
    obj.Dx0[noDataMask] = 0
    obj.Dy0[noDataMask] = 0
    obj.NoDataMask = noDataMask

    print("preprocessing")
    obj.WallisFilterWidth = preproc_filter_width
    obj.preprocess_filt_hps()
    obj.uniform_data_type()

    print("starting autoRIFT")
    obj.runAutorift()
    print("autoRIFT complete")

    # convert displacement to m
    obj.Dx_m = obj.Dx * 10
    obj.Dy_m = obj.Dy * 10
        
    return obj

In [8]:
os.makedirs(f'{proc_path}/Sentinel2', exist_ok=True)
os.makedirs(f'{proc_path}/Sentinel2/IC-OF_RESULTS', exist_ok=True)
os.makedirs(f'{proc_path}/Sentinel2/VELOCITY', exist_ok=True)
os.makedirs(f'{proc_path}/Sentinel2/DISPLACEMENT', exist_ok=True)

In [9]:
offset_dict = {}

for i, pair in enumerate(pairs):
    print(f'working on {i+1}/{len(pairs)}')
    obj = run_autoRIFT(pair)

    # interpolate to original dimensions 
    x_coords = obj.xGrid[0, :]
    y_coords = obj.yGrid[:, 0]
    
    # Create a mesh grid for the img dimensions
    x_coords_new, y_coords_new = np.meshgrid(
        np.arange(obj.I1.shape[1]),
        np.arange(obj.I1.shape[0])
    )

    # Perform bilinear interpolation using scipy.interpolate.interpn
    Dx_full = interpn((y_coords, x_coords), obj.Dx, (y_coords_new, x_coords_new), method="linear", bounds_error=False)
    Dy_full = interpn((y_coords, x_coords), obj.Dy, (y_coords_new, x_coords_new), method="linear", bounds_error=False)
    Dx_m_full = interpn((y_coords, x_coords), obj.Dx_m, (y_coords_new, x_coords_new), method="linear", bounds_error=False)
    Dy_m_full = interpn((y_coords, x_coords), obj.Dy_m, (y_coords_new, x_coords_new), method="linear", bounds_error=False)
    
    # open img 
    ds = xr.open_dataset(f'{proc_path}/T32TLR_20200812T103031_B08.tif')
    ds = ds.assign({'Dx':(['y', 'x'], Dx_full),
                     'Dy':(['y', 'x'], Dy_full),
                     'Dx_m':(['y', 'x'], Dx_m_full),
                     'Dy_m':(['y', 'x'], Dy_m_full)})
    ds['veloc_x'] = ds.Dx_m/((pd.to_datetime(pair[1]) - pd.to_datetime(pair[0])).days/365.25)
    ds['veloc_y'] = ds.Dy_m/((pd.to_datetime(pair[1]) - pd.to_datetime(pair[0])).days/365.25)

    offset_dict[f'{pair[0]}_{pair[1]}'] = ds
   
    # write to tifs
    ds.Dx.rio.to_raster(f'{proc_path}/Sentinel2/IC-OF_RESULTS/EW_{pair[0]}-{pair[1]}_raw.tif')
    ds.Dy.rio.to_raster(f'{proc_path}/Sentinel2/IC-OF_RESULTS/NS_{pair[0]}-{pair[1]}_raw.tif')
    ds.Dx_m.rio.to_raster(f'{proc_path}/Sentinel2/DISPLACEMENT/EW_{pair[0]}-{pair[1]}_disp_tot.tif')
    ds.Dy_m.rio.to_raster(f'{proc_path}/Sentinel2/DISPLACEMENT/NS_{pair[0]}-{pair[1]}_disp_tot.tif')
    ds.veloc_x.rio.to_raster(f'{proc_path}/Sentinel2/VELOCITY/EW_{pair[0]}-{pair[1]}_vel_mean.tif')
    ds.veloc_y.rio.to_raster(f'{proc_path}/Sentinel2/VELOCITY/NS_{pair[0]}-{pair[1]}_vel_mean.tif')

    print('--------------------------------')

working on 1/3
opening pair: 20200812-20200916
preprocessing
starting autoRIFT


/mnt/Backups/gbrench/sw/miniconda3/envs/insar/lib/python3.11/site-packages/isce/components/contrib/geo_autoRIFT/autoRIFT/autoRIFT.py:1413: RuntimeWarning: All-NaN slice encountered
  D = np.nanmedian(B,axis=0)
/mnt/Backups/gbrench/sw/miniconda3/envs/insar/lib/python3.11/site-packages/isce/components/contrib/geo_autoRIFT/autoRIFT/autoRIFT.py:1416: RuntimeWarning: All-NaN slice encountered
  C = np.nanmedian(D,axis=0, overwrite_input=True)
/mnt/Backups/gbrench/sw/miniconda3/envs/insar/lib/python3.11/site-packages/isce/components/contrib/geo_autoRIFT/autoRIFT/autoRIFT.py:1403: RuntimeWarning: All-NaN slice encountered
  C = np.nanmedian(B,axis=0, overwrite_input=True)
/mnt/Backups/gbrench/sw/miniconda3/envs/insar/lib/python3.11/site-packages/isce/components/contrib/geo_autoRIFT/autoRIFT/autoRIFT.py:1468: RuntimeWarning: divide by zero encountered in divide
  DxMadmin = np.ones(Dx.shape) / OverSampleRatio / SearchLimitX * 2;
/mnt/Backups/gbrench/sw/miniconda3/envs/insar/lib/python3.11/site

autoRIFT complete
--------------------------------
working on 2/3
opening pair: 20200812-20210812
preprocessing
starting autoRIFT


/mnt/Backups/gbrench/sw/miniconda3/envs/insar/lib/python3.11/site-packages/isce/components/contrib/geo_autoRIFT/autoRIFT/autoRIFT.py:1413: RuntimeWarning: All-NaN slice encountered
  D = np.nanmedian(B,axis=0)
/mnt/Backups/gbrench/sw/miniconda3/envs/insar/lib/python3.11/site-packages/isce/components/contrib/geo_autoRIFT/autoRIFT/autoRIFT.py:1416: RuntimeWarning: All-NaN slice encountered
  C = np.nanmedian(D,axis=0, overwrite_input=True)
/mnt/Backups/gbrench/sw/miniconda3/envs/insar/lib/python3.11/site-packages/isce/components/contrib/geo_autoRIFT/autoRIFT/autoRIFT.py:1403: RuntimeWarning: All-NaN slice encountered
  C = np.nanmedian(B,axis=0, overwrite_input=True)
/mnt/Backups/gbrench/sw/miniconda3/envs/insar/lib/python3.11/site-packages/isce/components/contrib/geo_autoRIFT/autoRIFT/autoRIFT.py:1468: RuntimeWarning: divide by zero encountered in divide
  DxMadmin = np.ones(Dx.shape) / OverSampleRatio / SearchLimitX * 2;
/mnt/Backups/gbrench/sw/miniconda3/envs/insar/lib/python3.11/site

autoRIFT complete
--------------------------------
working on 3/3
opening pair: 20200916-20210812
preprocessing
starting autoRIFT


/mnt/Backups/gbrench/sw/miniconda3/envs/insar/lib/python3.11/site-packages/isce/components/contrib/geo_autoRIFT/autoRIFT/autoRIFT.py:1413: RuntimeWarning: All-NaN slice encountered
  D = np.nanmedian(B,axis=0)
/mnt/Backups/gbrench/sw/miniconda3/envs/insar/lib/python3.11/site-packages/isce/components/contrib/geo_autoRIFT/autoRIFT/autoRIFT.py:1416: RuntimeWarning: All-NaN slice encountered
  C = np.nanmedian(D,axis=0, overwrite_input=True)
/mnt/Backups/gbrench/sw/miniconda3/envs/insar/lib/python3.11/site-packages/isce/components/contrib/geo_autoRIFT/autoRIFT/autoRIFT.py:1403: RuntimeWarning: All-NaN slice encountered
  C = np.nanmedian(B,axis=0, overwrite_input=True)
/mnt/Backups/gbrench/sw/miniconda3/envs/insar/lib/python3.11/site-packages/isce/components/contrib/geo_autoRIFT/autoRIFT/autoRIFT.py:1468: RuntimeWarning: divide by zero encountered in divide
  DxMadmin = np.ones(Dx.shape) / OverSampleRatio / SearchLimitX * 2;
/mnt/Backups/gbrench/sw/miniconda3/envs/insar/lib/python3.11/site

autoRIFT complete
--------------------------------


In [14]:
EW_raw_ds = xr.open_dataset(f'{proc_path}/Sentinel2/IC-OF_RESULTS/EW_20200916-20210812_raw.tif')
NS_raw_ds = xr.open_dataset(f'{proc_path}/Sentinel2/IC-OF_RESULTS/NS_20200916-20210812_raw.tif')

In [15]:
(EW_raw_ds.band_data*10).rio.to_raster(f'{proc_path}/Sentinel2/DISPLACEMENT/EW_20200916-20210812_disp_tot.tif')
(NS_raw_ds.band_data*10).rio.to_raster(f'{proc_path}/Sentinel2/DISPLACEMENT/NS_20200916-20210812_disp_tot.tif')